## PART 1

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
import html5lib
import lxml

In [3]:
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

In [4]:
from pandas.io.html import read_html

Importing the website

In [5]:
toronto=('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

Selecting the table

In [6]:
postal_codes=read_html (toronto, attrs={"class":"wikitable"})

Checking the 5 first lines of the table

In [7]:
postal_codes[0].head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Checking the number of rows and lines of the table

In [11]:
postal_codes[0].shape

(287, 3)

Deleting rows with not assigned borough

In [12]:
indexNames = postal_codes[0][ postal_codes[0]['Borough'] == 'Not assigned'].index
 
postal_codes[0].drop(indexNames , inplace=True)

In [13]:
postal_codes[0].shape

(210, 3)

Rename the table "to1"

In [14]:
to1= postal_codes[0]

In [15]:
to1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area.These rows will be combined into one row with the neighborhoods separated with a comma

In [16]:
to2= to1.groupby('Postcode').agg(lambda x: ','.join(x))

In [17]:
to2.shape

(103, 2)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [18]:
to2.loc[to2['Neighbourhood']=="Not assigned",'Neighbourhood']=to2.loc[to2['Neighbourhood']=="Not assigned",'Borough']

Removing the duplicate borough

In [22]:
to2['Borough']= to2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [24]:
to2.shape

(103, 2)